In [ ]:
!pip install scrapy -q
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from scrapy.linkextractors import LinkExtractor
import re
import pandas as pd

     |████████████████████████████████| 264 kB 6.9 MB/s 
     |████████████████████████████████| 3.1 MB 33.0 MB/s 
     |████████████████████████████████| 93 kB 1.9 MB/s 
     |████████████████████████████████| 251 kB 46.6 MB/s 
     |████████████████████████████████| 4.1 MB 45.5 MB/s 
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 47 kB 4.1 MB/s 
     |████████████████████████████████| 74 kB 1.6 MB/s 


In [ ]:
class scrapingSpider(scrapy.Spider):
    name = "kayak"
    villes =  ["Mont Saint Michel",
                "St Malo",
               "Bayeux",
               "Le Havre",
               "Rouen",
               "Paris",
               "Amiens",
               "Lille",
               "Strasbourg",
               "Chateau du Haut Koenigsbourg",
               "Colmar",
               "Eguisheim",
               "Besancon",
               "Dijon",
               "Annecy",
               "Grenoble",
               "Lyon",
               "Gorges du Verdon",
               "Bormes les Mimosas",
               "Cassis",
               "Marseille",
               "Aix en Provence",
               "Avignon",
               "Uzes",
               "Nimes",
               "Aigues Mortes",
               "Saintes Maries de la mer",
               "Collioure",
               "Carcassonne",
               "Ariege",
               "Toulouse",
               "Montauban",
               "Biarritz",
               "Bayonne",
               "La Rochelle"]
                           
    start_urls = ['https://www.booking.com/']

    def parse(self, response):
        for i in range(len(self.villes)):
            yield scrapy.FormRequest.from_response(
                response,
                formdata={'ss': self.villes[i]},
                callback=self.after_search,
                meta={'search': self.villes[i]}   
            )
            
    def after_search(self, response):
        results = response.css('div.b978843432')
        c = response.request
        location = " ".join(re.search(r'(?<=&ss=)[a-zA-Z\+]{2,30}', str(c)).group(0).split("+")) 
        for r in results[:]:
            hotel = {"location": location,
                     "name": r.css('div.fcab3ed991.a23c043802::text').get(),
                      "url": r.css('a.e13098a59f::attr(href)').get(),
                      "score" : r.css('div.b5cd09854e.d10a6220b4::text').get(),
                      "description" : r.css('div.d8eab2cf7f::text').get()}
            yield hotel
            
        url_booking = response.request.url
        if bool(re.search("&offset=[0-9]{1,10}", url_booking)):
            end = re.search(r'\d+$', url_booking)
            index = int(end.group(0)) + 25
            last_page = response.css('li.f32a99c8d1 button.fc63351294.f9c5690c58::text')[-1].get()
            if index > (int(last_page)*25):
                logging.info('No next page. Terminating crawling process.') 
                
            else:
                next_page = re.sub("&offset=[0-9]{1,10}","",url_booking)+"&offset="+str(index)
                yield response.follow(next_page, callback=self.after_search)
        else:
            index = 25
            next_page = url_booking+"&offset="+ str(index)
            yield response.follow(next_page, callback=self.after_search)    

In [ ]:
filename = "hotels.json"

if filename in os.listdir('/root'):
        os.remove('/root' + filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        'result_' + filename : {"format": "json"}
    }
})

process.crawl(scrapingSpider)
process.start()

2022-07-30 16:03:55 [scrapy.utils.log] INFO: Scrapy 2.6.2 started (bot: scrapybot)
2022-07-30 16:03:55 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.5 5 Jul 2022), cryptography 37.0.4, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-07-30 16:03:55 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'}
2022-07-30 16:03:55 [scrapy.extensions.telnet] INFO: Telnet Password: 11145ee70f08c27b
2022-07-30 16:03:55 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.Lo